# Laden von mehrkanal / mehrpositions Ordnern mit tif-Dateien
Einige Mikroskope schreiben Bilddaten scheibchenweise als tif-Dateien auf die Festplatte. Man findet dann viele Dateien in diesen Ordnern, die zum Beispiel `image_z03_ch01.tif` heißen. Um diese Ordner zu laden, müssen Sie typischerweise Details kennen, wie zum Beispiel wie viele z-Scheiben und wie viele Kanäle aufgenommen wurden, oder ob die Bilddaten mehrere Zeitpunkte haben oder nicht.

Siehe auch [diese Diskussion](https://forum.image.sc/t/reading-2d-images-as-a-multi-channel-3d-stack-in-python/75460).

In [1]:
from skimage import io
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np

import shutil
import os

path = "../../data/tif_folder/"

Zu Demonstrationszwecken erstellen wir einfach einen solchen Ordner mit Dummy-Bilddaten.

In [2]:
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
for z in range(1,7):
    for c in range(1,5):
        #image = io.imread("c:/structure/data/blobs.tif")
        #io.imsave(f"c:/structure/data/images/r01c01f34p0{z}-ch0{c}t01.tiff", image)
        
        shutil.copy("../../data/blobs.tif",
                    path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff")

Um einen Überblick zu bekommen, können wir die Dateinamen im Ordner ausgeben.

In [4]:
for file in os.listdir(path):
    print(file)

image_z01-ch01.tiff
image_z01-ch02.tiff
image_z01-ch03.tiff
image_z01-ch04.tiff
image_z02-ch01.tiff
image_z02-ch02.tiff
image_z02-ch03.tiff
image_z02-ch04.tiff
image_z03-ch01.tiff
image_z03-ch02.tiff
image_z03-ch03.tiff
image_z03-ch04.tiff
image_z04-ch01.tiff
image_z04-ch02.tiff
image_z04-ch03.tiff
image_z04-ch04.tiff
image_z05-ch01.tiff
image_z05-ch02.tiff
image_z05-ch03.tiff
image_z05-ch04.tiff
image_z06-ch01.tiff
image_z06-ch02.tiff
image_z06-ch03.tiff
image_z06-ch04.tiff


scikit-image bietet eine [`imread_collection`](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread_collection) zum Laden von Dateien, die zu einem Muster passen, z.B. solche, die `*` enthalten.

In [5]:
im_collection = io.imread_collection(path + "*")
im_collection

Sie können diese Sammlung von Bildern in einen auf numpy-array basierenden Bildstapel umwandeln. Leider ist die Anzahl der z-Scheiben und Kanäle zu diesem Zeitpunkt unbekannt.

In [6]:
image_stack = im_collection.concatenate()
image_stack.shape

(24, 254, 256)

Wenn Sie die Anzahl der z-Scheiben und Kanäle kennen, können Sie das Bild mit [`reshape`]() in ein 3D+ch oder 4D Bild umformen.

In [7]:
num_channels = 4
num_z_slices = 6
image4d = np.reshape(image_stack, (num_channels, num_z_slices, image_stack.shape[-2], image_stack.shape[-1]))
image4d.shape

(4, 6, 254, 256)

Alternativ können Sie auch Ihre eigenen for-Schleifen zum Laden der Daten von der Festplatte erstellen. Dies gibt Ihnen etwas mehr Freiheit, z.B. beim Sortieren von Scheiben und Kanälen in die verwendeten Dimensionen.

In [8]:
num_channels = 4
num_z_slices = 6

image4d_loaded = np.asarray([
    [io.imread(path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff") for c in range(1, 5)]
    for z in range(1, 7)
])
image4d_loaded.shape

(6, 4, 254, 256)